In [ ]:
from h5_data import load_train_data, load_test_data
import numpy as np
from key_mappings import js_keymap
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
data = np.load('desktop/desktop_dev_set.npy', allow_pickle=True).item()

In [ ]:
keys = list(data.keys())
print(keys[:20])
user_key = '0HdCtvLt' # keys[1]
print(data[keys[1]].keys())
session_key = '1ArY0fEb' # list(data[user_key].keys())[3]

In [ ]:
sample = data[user_key][session_key]
print(len(sample))

for arr in sample:
    letter = (arr[2])
    print(letter, end=' ')
print()
for arr in sample:
    letter = js_keymap[arr[2]]
    if arr[2] == 16:
        letter = '↑'
    elif arr[2] == 32:
        letter = '␠'
    elif arr[2] == 8:
        letter = '⌫'
    elif arr[2] == 13:
        letter = '⏎'
    elif arr[2] == 46:
        letter = '⌦'
    elif arr[2] == 20:
        letter = '⇪'
    print(letter, end='')

In [ ]:
def preprocess(sample):
    data = sample
    data[:, :2] = (data[:, :2] - data[0][0])
    duration = np.abs(data[:, 1] - data[:, 0])  # fix duration of keypress. ~349 items are broken
    diff_press = np.diff(data[:, 0], prepend=data[0, 0])
    data = np.column_stack((diff_press, duration, data[:, 2]))
    return data

preprocess(sample.copy())

In [ ]:
diffs = defaultdict(int)
durations = defaultdict(int)
sample_lengths = defaultdict(int)

for user, user_data in data.items():
    for session, sample in user_data.items():
        preprocessed = preprocess(sample.copy())
        l = len(preprocessed)
        sample_lengths[l] += 1
        diff = np.median(preprocessed[:, 0])
        diffs[diff] += 1
        dur = np.median(preprocessed[:, 1])
        durations[dur] += 1


In [ ]:
plt.bar(durations.keys(), durations.values(), log=True)
plt.title('Median duration of keypresses, ms')
plt.show()

In [ ]:
plt.bar(sample_lengths.keys(), sample_lengths.values(), log=True)
plt.title('Length of samples, # of keypresses')
plt.show()

In [ ]:
plt.bar(diffs.keys(), diffs.values(), log=True)
plt.title('Median difference between keypresses, ms')
plt.show()